In [2]:
import pandas as pd
import re
import requests
from io import BytesIO
import numpy as np
from typing import Optional

In [3]:
file_id = "1LvhYDiOFFoK_LedDQZa6_dQziXI5YqLy6oejwDJhLWQ"
sheet_name = "HIV2013"

url = (
    f"https://docs.google.com/spreadsheets/d/{file_id}/export"
    f"?format=xlsx&sheet={sheet_name}"
)

main_df = pd.read_excel(url, sheet_name=sheet_name)

In [4]:
new_header = main_df.iloc[1]
main_df = main_df.iloc[3:]
main_df.columns = new_header
main_df = main_df.reset_index(drop=True)

main_df.head()

1,Country,WHO Region,Population,Geographical Region,WHO Group,Total DALY,Adult DALYs,Children DALYs,Retention Rate,Retention Rate (ADULT),...,"Reported number of people receiving antiretroviral therapy, month and year of report",Cleaned coverage,Cleaned number of people receiving antiretroviral therapy,http://apps.who.int/gho/data/view.main.23300PA,Estimated antiretroviral therapy coverage among children,Estimated number of children needing antiretroviral therapy based on WHO methods,Reported number of children receiving antiretroviral therapy,Cleaned Coverage (child),Cleaned Estimated Needing (child),Cleaned Receiving Number (child)
0,Afghanistan,EMR,31731688,"East, South and South-East Asia",A,6800.823993,5935.007821,865.816172,24,23,...,2014-12-01 00:00:00,0.04,281,Afghanistan,4 [3 - 8],&lt;500 [&lt;500-&lt;1000],16,0.04,500,16
1,Albania,EUR,2895092,Europe and Central Asia,A,85.119658,83.573072,1.546587,97.14,97.14,...,2014-12-01 00:00:00,NaN,354,Albania,No data,No data,No data,NaN,NaN,NaN
2,Algeria,AFR,38338562,Middle East and North Africa,A,4116.867327,4059.284488,57.582839,90,90,...,2014-12-01 00:00:00,0.57,6020,Algeria,No data,No data,No data,NaN,NaN,NaN
3,American Samoa,WPR,55307,NaN,A,108.866657,101.089193,7.777464,97.14,97.14,...,No data,NaN,NaN,Andorra,No data,No data,No data,NaN,NaN,NaN
4,Andorra,EUR,80788,NaN,A,61.841325,61.769814,0.071511,97.14,97.14,...,No data,NaN,76666,Angola,No data,No data,No data,NaN,NaN,NaN


In [5]:
main_df.columns

Index([                                                                             'Country',
                                                                                 'WHO Region',
                                                                                 'Population',
                                                                        'Geographical Region',
                                                                                  'WHO Group',
                                                                                 'Total DALY',
                                                                                'Adult DALYs',
                                                                             'Children DALYs',
                                                                             'Retention Rate',
                                                                     'Retention Rate (ADULT)',
                                                  

In [6]:
import re
import numpy as np
import pandas as pd

# ---------- Excel column letters -> 0-based position ----------
def _col_idx(col_letters: str) -> int:
    col_letters = col_letters.strip().upper()
    n = 0
    for ch in col_letters:
        n = n * 26 + (ord(ch) - ord("A") + 1)
    return n - 1

def _to_float(x):
    """Robust float conversion; handles %, commas, blanks."""
    if pd.isna(x):
        return np.nan
    if isinstance(x, str):
        s = x.strip().replace(",", "")
        if s == "" or s.lower() == "nan":
            return np.nan
        if s.endswith("%"):
            try:
                return float(s[:-1]) / 100.0
            except Exception:
                return np.nan
        try:
            return float(s)
        except Exception:
            return np.nan
    try:
        return float(x)
    except Exception:
        return np.nan

def _drug_token_from_colname(colname) -> str:
    """
    2013 columns are like 'Impact of 3TC'. Regimen strings contain '3TC'.
    """
    s = str(colname).strip()
    s = re.sub(r"(?i)^\s*impact\s+of\s+", "", s).strip()
    return s

def _contains_drug(regimen_text, drug_token: str) -> bool:
    if pd.isna(regimen_text):
        return False
    reg = str(regimen_text).upper().replace(" ", "")
    tok = str(drug_token).upper().replace(" ", "")
    return tok != "" and tok in reg

def _regimen_size(regimen_text) -> float:
    """
    Excel uses /3 or /4 depending on how many drugs are in the regimen.
    We infer from the regimen string (e.g., 'AZT + 3TC + NVP' -> 3).
    """
    if pd.isna(regimen_text):
        return 3.0
    s = str(regimen_text)
    parts = [p.strip() for p in re.split(r"\+", s) if p.strip() and p.strip().lower() != "nan"]
    if len(parts) >= 2:
        return float(len(parts))
    # fallback if someone used commas
    parts2 = [p.strip() for p in s.split(",") if p.strip() and p.strip().lower() != "nan"]
    if len(parts2) >= 2:
        return float(len(parts2))
    return 3.0

def _impact_term(base, const, coef1, var, coef2, denom) -> float:
    """
    Matches pattern:
      base * const * coef1 * var * coef2 / (1 - const*coef1*var*coef2) / denom
    """
    if denom == 0 or pd.isna(denom):
        return 0.0
    if any(pd.isna(v) for v in [base, const, coef1, var, coef2]):
        return 0.0
    prod = const * coef1 * var * coef2
    d = 1.0 - prod
    if d == 0:
        return 0.0
    return (base * prod / d) / denom


def compute_impact_score_2013_dynamic(
    main_df: pd.DataFrame,
    drug_col_index: int,
    output_col: Optional[str] = None,
    excel_row0: int = 5,
    b_regimen_excel_rows = range(8, 21),
    a_regimen_excel_rows = range(27, 38),
    first_line_reg_col: str = "AS",
    second_line_reg_col: str = "AX",
    debug: bool = False,
    debug_show_first_n_rows: int = 3,
):
    """
    Dynamic HIV2013 impact score calculator for ONE drug column.
    No pasted formulas; it searches the regimen tables like your 2017 logic.
    """

    # --- indices for the country-row inputs (same as your 2017 notebook positions) ---
    idx_E = _col_idx("E")   # WHO Group (A/B)
    idx_G = _col_idx("G")   # Adult DALYs
    idx_H = _col_idx("H")   # Children DALYs
    idx_I = _col_idx("I")   # Retention Rate (overall) used in normalization
    idx_Q = _col_idx("Q")   # adult coverage factor (matches your 2017 code: iloc[16])
    idx_T = _col_idx("T")   # child coverage factor (matches your 2017 code: iloc[19])

    # --- constants columns ---
    idx_AP = _col_idx("AP")
    idx_AQ = _col_idx("AQ")

    # --- regimen block start columns (we’ll use offsets from the regimen-name col) ---
    idx_FL_REG = _col_idx(first_line_reg_col)   # AS
    idx_SL_REG = _col_idx(second_line_reg_col)  # AX

    # First-line: AS (reg), AT (adult%), AU (adult eff), AV (child%), AW (child eff)
    idx_AT = idx_FL_REG + 1
    idx_AU = idx_FL_REG + 2
    idx_AV = idx_FL_REG + 3
    idx_AW = idx_FL_REG + 4

    # Second-line: AX (reg), AY (adult%), AZ (adult eff), BA (child%), BB (child eff)
    idx_AY = idx_SL_REG + 1
    idx_AZ = idx_SL_REG + 2
    idx_BA = idx_SL_REG + 3
    idx_BB = idx_SL_REG + 4

    # --- excel row -> df index ---
    def r(excel_row: int) -> int:
        return excel_row - excel_row0

    b_rows = [r(x) for x in b_regimen_excel_rows if 0 <= r(x) < len(main_df)]
    a_rows = [r(x) for x in a_regimen_excel_rows if 0 <= r(x) < len(main_df)]

    # --- pull constants from absolute rows (row numbers match the Excel formula) ---
    # B constants (AP)
    AP5  = _to_float(main_df.iat[r(5),  idx_AP])
    AP6  = _to_float(main_df.iat[r(6),  idx_AP])
    AP10 = _to_float(main_df.iat[r(10), idx_AP])
    AP11 = _to_float(main_df.iat[r(11), idx_AP])

    # A constants (AQ)
    AQ5  = _to_float(main_df.iat[r(5),  idx_AQ])
    AQ6  = _to_float(main_df.iat[r(6),  idx_AQ])
    AQ10 = _to_float(main_df.iat[r(10), idx_AQ])
    AQ11 = _to_float(main_df.iat[r(11), idx_AQ])

    colname = main_df.columns[drug_col_index]
    drug_token = _drug_token_from_colname(colname)

    if output_col is None:
        output_col = f"Computed Impact Score ({drug_token})"

    if debug:
        print("---- DEBUG (constants) ----")
        print(f"Drug column name: {colname!r}  -> token used in search: {drug_token!r}")
        print(f"AP5={AP5}, AP6={AP6}, AP10={AP10}, AP11={AP11}")
        print(f"AQ5={AQ5}, AQ6={AQ6}, AQ10={AQ10}, AQ11={AQ11}")
        print(f"First-line regimen col: {first_line_reg_col} (idx {idx_FL_REG})")
        print(f"Second-line regimen col: {second_line_reg_col} (idx {idx_SL_REG})")
        print("---------------------------")

    scores = []

    for i in range(len(main_df)):
        row = main_df.iloc[i]

        try:
            who = str(row.iloc[idx_E]).strip().upper()
            G = _to_float(row.iloc[idx_G])
            H = _to_float(row.iloc[idx_H])
            Q = _to_float(row.iloc[idx_Q])
            T = _to_float(row.iloc[idx_T])
            I = _to_float(row.iloc[idx_I])
        except Exception:
            scores.append(0.0)
            continue

        # Choose regimen block + constants by WHO group
        if who == "B":
            rows_idx = b_rows
            k_FL_adult, k_FL_child = AP5, AP10
            k_SL_adult, k_SL_child = AP6, AP11
        else:
            rows_idx = a_rows
            k_FL_adult, k_FL_child = AQ5, AQ10
            k_SL_adult, k_SL_child = AQ6, AQ11

        total = 0.0
        fl_hits = 0
        sl_hits = 0

        for rr in rows_idx:
            # ---- First-line block (regimen name in AS) ----
            fl_reg = main_df.iat[rr, idx_FL_REG]
            if _contains_drug(fl_reg, drug_token):
                denom = _regimen_size(fl_reg)
                AT = _to_float(main_df.iat[rr, idx_AT])
                AU = _to_float(main_df.iat[rr, idx_AU])
                AV = _to_float(main_df.iat[rr, idx_AV])
                AW = _to_float(main_df.iat[rr, idx_AW])

                total += _impact_term(G, k_FL_adult, AT, Q, AU, denom)
                total += _impact_term(H, k_FL_child, AV, T, AW, denom)
                fl_hits += 1

            # ---- Second-line block (regimen name in AX) ----
            sl_reg = main_df.iat[rr, idx_SL_REG]
            if _contains_drug(sl_reg, drug_token):
                denom2 = _regimen_size(sl_reg)
                AY = _to_float(main_df.iat[rr, idx_AY])
                AZ = _to_float(main_df.iat[rr, idx_AZ])
                BA = _to_float(main_df.iat[rr, idx_BA])
                BB = _to_float(main_df.iat[rr, idx_BB])

                total += _impact_term(G, k_SL_adult, AY, Q, AZ, denom2)
                total += _impact_term(H, k_SL_child, BA, T, BB, denom2)
                sl_hits += 1

        # Normalize like Excel: /(100/(100-I))
        try:
            norm = 100.0 / (100.0 - I)
            result = total / norm if norm != 0 and not pd.isna(norm) else 0.0
        except Exception:
            result = 0.0

        if debug and i < debug_show_first_n_rows:
            print(f"[DEBUG row {i}] WHO={who}, FL_hits={fl_hits}, SL_hits={sl_hits}, result={result}")

        scores.append(result)

    main_df[output_col] = scores
    return main_df

In [7]:
main_df = compute_impact_score_2013_dynamic(
    main_df,
    drug_col_index=24,   # Impact of 3TC
    output_col="Computed Impact Score (3TC)",
    debug=True
)

main_df[["Country", "WHO Group", "Impact of 3TC", "Computed Impact Score (3TC)"]].head(10)

---- DEBUG (constants) ----
Drug column name: 'Impact of 3TC'  -> token used in search: '3TC'
AP5=0.8543, AP6=0.1457, AP10=0.821, AP11=0.179
AQ5=0.9624, AQ6=0.0376, AQ10=0.9655, AQ11=0.0345
First-line regimen col: AS (idx 44)
Second-line regimen col: AX (idx 49)
---------------------------
[DEBUG row 0] WHO=A, FL_hits=8, SL_hits=7, result=49.430682421542514
[DEBUG row 1] WHO=A, FL_hits=8, SL_hits=7, result=0.0
[DEBUG row 2] WHO=A, FL_hits=8, SL_hits=7, result=59.872738874983256


1,Country,WHO Group,Impact of 3TC,Computed Impact Score (3TC)
0,Afghanistan,A,49.430682,49.430682
1,Albania,A,0,0.000000
2,Algeria,A,59.872739,59.872739
3,American Samoa,A,0.274228,0.274228
4,Andorra,A,0.159732,0.159732
5,Angola,A,4893.790271,4893.790270
6,Anguilla,B,0,0.000000
7,Antigua and Barbuda,B,27.364383,27.110788
8,Argentina,B,267.959938,265.744551
9,Armenia,A,7.904949,7.904949


In [8]:
impact_cols = [
    (i, c) for i, c in enumerate(main_df.columns)
    if isinstance(c, str) and c.strip().lower().startswith("impact of ")
]

for idx, col in impact_cols:
    drug = _drug_token_from_colname(col)
    main_df = compute_impact_score_2013_dynamic(
        main_df,
        drug_col_index=idx,
        output_col=f"Computed Impact Score ({drug})",
        debug=False
    )

In [9]:
main_df.columns

Index([                                                                             'Country',
                                                                                 'WHO Region',
                                                                                 'Population',
                                                                        'Geographical Region',
                                                                                  'WHO Group',
                                                                                 'Total DALY',
                                                                                'Adult DALYs',
                                                                             'Children DALYs',
                                                                             'Retention Rate',
                                                                     'Retention Rate (ADULT)',
                                                  

In [10]:
import numpy as np
import pandas as pd

# --- your computed drug impact columns ---
computed_drug_cols_wanted = [
    'Computed Impact Score (3TC)',
    'Computed Impact Score (ABC)',
    'Computed Impact Score (AZT)',
    'Computed Impact Score (ddl)',
    'Computed Impact Score (d4T)',
    'Computed Impact Score (EFV)',
    'Computed Impact Score (FTC)',
    'Computed Impact Score (LPV/r)',
    'Computed Impact Score (NVP)',
    'Computed Impact Score (TDF)',
    'Computed Impact Score (ATV/r)',
]

# only keep the ones that exist right now
computed_drug_cols = [c for c in computed_drug_cols_wanted if c in main_df.columns]
missing = [c for c in computed_drug_cols_wanted if c not in main_df.columns]
if missing:
    print("⚠️ Missing computed columns (not included in overall):")
    for m in missing:
        print("   -", m)

# ensure numeric
for c in computed_drug_cols:
    main_df[c] = pd.to_numeric(main_df[c], errors="coerce")

# per-country overall computed impact
main_df["Computed Overall Treatment Impact"] = main_df[computed_drug_cols].sum(axis=1, skipna=True)


In [11]:
global_computed_total = main_df["Computed Overall Treatment Impact"].sum(skipna=True)
global_actual_total = pd.to_numeric(main_df["Overall Treatment Impact"], errors="coerce").sum(skipna=True)

print("Global totals (sum across countries):")
print("  Computed:", global_computed_total)
print("  Actual  :", global_actual_total)
print("  Diff    :", global_computed_total - global_actual_total)


Global totals (sum across countries):
  Computed: 5455865.573768298
  Actual  : 5469262.43103413
  Diff    : -13396.857265831903


In [12]:
from IPython.display import display

pairs = [
    ("3TC",   "Impact of 3TC",   "Computed Impact Score (3TC)"),
    ("ABC",   "Impact of ABC",   "Computed Impact Score (ABC)"),
    ("AZT",   "Impact of AZT",   "Computed Impact Score (AZT)"),
    ("ddl",   "Impact of ddl",   "Computed Impact Score (ddl)"),
    ("d4T",   "Impact of d4T",   "Computed Impact Score (d4T)"),
    ("EFV",   "Impact of EFV",   "Computed Impact Score (EFV)"),
    ("FTC",   "Impact of FTC",   "Computed Impact Score (FTC)"),
    ("LPV/r", "Impact of LPV/r", "Computed Impact Score (LPV/r)"),
    ("NVP",   "Impact of NVP",   "Computed Impact Score (NVP)"),
    ("TDF",   "Impact of TDF",   "Computed Impact Score (TDF)"),
    ("ATV/r", "Impact of ATV/r", "Computed Impact Score (ATV/r)"),
    ("Overall", "Overall Treatment Impact", "Computed Overall Treatment Impact"),
]

# Keep only pairs where both columns exist
pairs_existing = []
for drug, a, c in pairs:
    if a in main_df.columns and c in main_df.columns:
        pairs_existing.append((drug, a, c))
    else:
        print(f"⚠️ Skipping {drug}: missing column(s):",
              [x for x in [a, c] if x not in main_df.columns])

# Make sure numeric
for _, a, c in pairs_existing:
    main_df[a] = pd.to_numeric(main_df[a], errors="coerce")
    main_df[c] = pd.to_numeric(main_df[c], errors="coerce")

# Build comparison table
out = main_df[["Country", "WHO Group"]].copy()

for drug, a, c in pairs_existing:
    out[f"{drug} | Actual"] = main_df[a]
    out[f"{drug} | Computed"] = main_df[c]
    out[f"{drug} | Diff (Comp-Act)"] = main_df[c] - main_df[a]

display(out.head(50))


1,Country,WHO Group,3TC | Actual,3TC | Computed,3TC | Diff (Comp-Act),ABC | Actual,ABC | Computed,ABC | Diff (Comp-Act),AZT | Actual,AZT | Computed,...,NVP | Diff (Comp-Act),TDF | Actual,TDF | Computed,TDF | Diff (Comp-Act),ATV/r | Actual,ATV/r | Computed,ATV/r | Diff (Comp-Act),Overall | Actual,Overall | Computed,Overall | Diff (Comp-Act)
0,Afghanistan,A,49.430682,49.430682,1.542517e-09,0.544295,0.578202,0.033907,25.104365,25.104365,...,-2.503185e-09,11.669486,11.437419,-0.232067,0.000000,0.000000,0.000000e+00,164.190528,163.528234,-0.662294
1,Albania,A,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
2,Algeria,A,59.872739,59.872739,-5.016744e-09,0.105490,0.154836,0.049346,29.884050,29.884050,...,-2.607123e-10,15.064007,14.770302,-0.293705,0.000000,0.000000,0.000000e+00,199.971599,199.139829,-0.831770
3,American Samoa,A,0.274228,0.274228,4.939399e-11,0.001353,0.001567,0.000215,0.137633,0.137633,...,-5.453166e-12,0.067874,0.066541,-0.001334,0.000000,0.000000,0.000000e+00,0.914649,0.910862,-0.003787
4,Andorra,A,0.159732,0.159732,1.719336e-11,0.000301,0.000438,0.000137,0.079116,0.079116,...,3.407907e-11,0.041463,0.040648,-0.000815,0.000000,0.000000,0.000000e+00,0.535254,0.532946,-0.002308
5,Angola,A,4893.790271,4893.790270,-1.131882e-06,96.666083,98.669933,2.003850,2629.698975,2629.698975,...,-4.726276e-07,911.366159,893.487035,-17.879124,0.000000,0.000000,0.000000e+00,15934.519040,15882.885518,-51.633522
6,Anguilla,B,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
7,Antigua and Barbuda,B,27.364383,27.110788,-2.535951e-01,2.565031,2.558952,-0.006079,15.684962,15.684962,...,1.680847e-10,14.074069,14.135883,0.061814,3.780733,3.780733,-2.231300e-10,93.954045,97.139393,3.185347
8,Argentina,B,267.959938,265.744551,-2.215387e+00,23.247982,24.704143,1.456161,154.531281,154.531280,...,-2.387424e-09,136.188698,137.611644,1.422946,36.667005,36.667005,1.735302e-09,918.226336,950.541203,32.314867
9,Armenia,A,7.904949,7.904949,9.049543e-10,0.014995,0.022011,0.007016,3.889442,3.889442,...,6.976020e-10,2.107273,2.065563,-0.041710,0.000000,0.000000,0.000000e+00,26.565144,26.447031,-0.118113


In [13]:
pd.set_option('display.max_rows', None)
main_df

1,Country,WHO Region,Population,Geographical Region,WHO Group,Total DALY,Adult DALYs,Children DALYs,Retention Rate,Retention Rate (ADULT),...,Computed Impact Score (AZT),Computed Impact Score (ddl),Computed Impact Score (d4T),Computed Impact Score (EFV),Computed Impact Score (FTC),Computed Impact Score (LPV/r),Computed Impact Score (NVP),Computed Impact Score (TDF),Computed Impact Score (ATV/r),Computed Overall Treatment Impact
0,Afghanistan,EMR,31731688,"East, South and South-East Asia",A,6800.823993,5935.007821,865.816172,24,23,...,25.104365,1.198146e-01,17.482916,14.619437,4.892582,1.145116,38.717700,1.143742e+01,0.000000,1.635282e+02
1,Albania,EUR,2895092,Europe and Central Asia,A,85.119658,83.573072,1.546587,97.14,97.14,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
2,Algeria,AFR,38338562,Middle East and North Africa,A,4116.867327,4059.284488,57.582839,90,90,...,29.884050,1.054902e-01,21.682692,17.732202,6.320998,1.261014,47.355506,1.477030e+01,0.000000,1.991398e+02
3,American Samoa,WPR,55307,NaN,A,108.866657,101.089193,7.777464,97.14,97.14,...,0.137633,5.479896e-04,0.098397,0.081527,0.028473,0.006007,0.215940,6.654065e-02,0.000000,9.108617e-01
4,Andorra,EUR,80788,NaN,A,61.841325,61.769814,0.071511,97.14,97.14,...,0.079116,2.934988e-04,0.057757,0.048636,0.017398,0.003501,0.125425,4.064775e-02,0.000000,5.329457e-01
5,Angola,AFR,25998340,Sub-Saharan Africa,A,670361.2384,491708.8564,178652.382,92,97.14,...,2629.698975,1.350499e+01,1680.215377,1269.761456,381.604334,105.858828,3916.294324,8.934870e+02,0.000000,1.588289e+04
6,Anguilla,AMR,13864,NaN,B,0,NaN,NaN,97.14,97.14,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
7,Antigua and Barbuda,AMR,97824,NaN,B,387.712324,368.27658,19.435744,6,97.14,...,15.684962,4.147307e-02,0.000000,20.166079,5.227537,4.462682,3.970303,1.413588e+01,3.780733,9.713939e+01
8,Argentina,AMR,42539925,Latin America and the Caribbean,B,87981.60546,83656.7217,4324.883763,97.14,97.14,...,154.531280,4.044171e-01,0.000000,198.488932,50.698099,43.302818,38.388312,1.376116e+02,36.667005,9.505412e+02
9,Armenia,EUR,2893509,Europe and Central Asia,A,1344.853434,1112.493971,232.359463,84,84,...,3.889442,1.499504e-02,2.854573,2.462878,0.884258,0.178995,6.169367,2.065563e+00,0.000000,2.644703e+01


In [14]:
main_df.to_csv("updated_HIV2013_nb.csv", index=False)
print("File saved as updated_HIV2013_nb.csv")

File saved as updated_HIV2013_nb.csv
